# L2: DSPy Programming - Signatures and Modules

#### Set up API key

In [43]:
from dotenv import load_dotenv

load_dotenv()

True

#### Configure the LM

In [44]:
import dspy

dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

### Use DSPy built-in Module to build a Sentiment Classifier

In [45]:
class SentimentClassifier(dspy.Signature):
    """Classify the Sentiment of a text"""

    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

In [46]:
str_signature = dspy.make_signature("text -> sentiment")

#### Create a module to interact with the LM

In [47]:
predict = dspy.Predict(SentimentClassifier)

In [48]:
output = predict(text="I am feeling very Happy!")
print(output)

Prediction(
    sentiment='9'
)


In [49]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is: {output['sentiment']}")

The sentiment is: 9
The sentiment is: 9


In [50]:
dspy.configure(lm=dspy.LM("openai/gpt-4o"))
print(predict(text="I am feeling very much Frankfurter Bahnhofsviertel").sentiment)

5


In [51]:
dspy.configure(lm=dspy.LM("openai/gpt-4o-mini"))

#### Wait, where is my prompt?

In [52]:
dspy.inspect_history(n=1)





[2025-10-23T15:53:35.676357]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `sentiment` (str): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "sentiment": "{sentiment}"
}
In adhering to this structure, your objective is: 
        Classify the Sentiment of a text


User message:

[[ ## text ## ]]
I am feeling very much Frankfurter Bahnhofsviertel

Respond with a JSON object in the following order of fields: `sentiment`.


Response:

{"sentiment":"5"}







#### Try a different built-in module

In [53]:
cot = dspy.ChainOfThought(SentimentClassifier)
output = cot(text="I am feeling much like frankfurter bahnhofsviertel")
print(output)

Prediction(
    reasoning="The phrase 'I am feeling much like frankfurter bahnhofsviertel' suggests a sense of confusion or disorientation, as it references a specific location known for its chaotic atmosphere. This implies a negative sentiment, as it does not convey happiness or positivity.",
    sentiment='3'
)


In [54]:
dspy.configure(adapter=dspy.JSONAdapter())

In [55]:
dspy.inspect_history(n=1)





[2025-10-23T15:53:38.501554]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str): 
2. `sentiment` (str): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "sentiment": "{sentiment}"
}
In adhering to this structure, your objective is: 
        Classify the Sentiment of a text


User message:

[[ ## text ## ]]
I am feeling much like frankfurter bahnhofsviertel

Respond with a JSON object in the following order of fields: `reasoning`, then `sentiment`.


Response:

{"reasoning":"The phrase 'I am feeling much like frankfurter bahnhofsviertel' suggests a sense of confusion or disorientation, as

### Build a Program with Custom Module

In [70]:
class QuestionGenerator(dspy.Signature):
    """Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name if you think the signal is enough. You should never ask the same question in the past_questions"""

    # Inputs
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[str] = dspy.InputField(desc="past answers")
    # Outputs
    new_question: str = dspy.OutputField(
        desc="new question that can help narrow down the celebrity name"
    )
    guess_made: str = dspy.OutputField(
        desc="If the new_question is the celebrity name guess, set to True, if it is still a general question, set to False"
    )


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""

    # Inputs
    correct_celebrity_name: str = dspy.InputField(
        desc="the celebrity name in user's mind"
    )
    final_guessor_question: str = dspy.InputField(
        desc="the final guess or question the guessor made"
    )
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[str] = dspy.InputField(desc="past answers")

    # Outputs
    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was done well and what can be improved"
    )


def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")


class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions, past_answers=past_answers
            )
            answer = str(ask(f"{question.new_question}").lower() == "y")
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made == "True" and answer == "True":
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=input(
                "Please type in the name of the celebrity in your mind and press enter ..."
            ),
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)


In [71]:
celebrity_guess = CelebrityGuess()

In [72]:
celebrity_guess()

Oops, I couldn't guess it right.
The guessing process was fairly systematic, with a clear focus on identifying the celebrity's profession. The initial questions effectively narrowed down the field to sports, and the distinction between individual and team sports was useful. However, the final question was too broad and did not consider the specific sport of skateboarding. In future guessing games, it would be beneficial to ask more specific questions related to the known sports of the celebrity, especially after confirming they are involved in an individual sport. This would help in making a more accurate final guess.


In [73]:
dspy.inspect_history(n=2)





[2025-10-23T16:10:13.604815]

System message:

Your input fields are:
1. `past_questions` (str): past questions asked
2. `past_answers` (str): past answers
Your output fields are:
1. `reasoning` (str): 
2. `new_question` (str): new question that can help narrow down the celebrity name
3. `guess_made` (str): If the new_question is the celebrity name guess, set to True, if it is still a general question, set to False
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## past_questions ## ]]
{past_questions}

[[ ## past_answers ## ]]
{past_answers}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "new_question": "{new_question}",
  "guess_made": "{guess_made}"
}
In adhering to this structure, your objective is: 
        Generate a yes or no question in order to guess the celebrity name in users' mind. You can ask in general or directly guess the name

#### Save and Load

In [77]:
from pathlib import Path

save_path = Path("dspy_program/celebrity.json")
save_path.parent.mkdir()
save_path.touch()
celebrity_guess.save("dspy_program/celebrity.json", save_program=False)

In [79]:
save_path = Path("dspy_program/celebrity")
save_path.mkdir()
celebrity_guess.save(save_path, save_program=True)

FileExistsError: [Errno 17] File exists: 'dspy_program/celebrity'